In [3]:
class A(object):
    pass

A.__bases__

(object,)

In [5]:
class A(metaclass=type):
    pass
A.__bases__

(object,)

In [6]:
class C(object):
    attr = 10
C = type('C', (object,), {'attr': 10})

In [9]:
def __init__(self, value):
    self.value = value
def add(self, value):
    self.value += value
C = type('C', (object,), {'attr': 10, 
                          '__init__': __init__, 
                          'add': add})

In [12]:
c = C(10)
# dir(c)

In [20]:
class MyMeta(type):
    def __str__(cls):
        return 'hello'
class A(metaclass=MyMeta):
    pass

In [21]:
str(A)

'hello'

In [27]:
class MyMeta(type):
    def __new__(meta, name, bases, cdict):
        print(meta)
        print(name)
        print(bases)
        print(cdict)
        return super().__new__(meta, name, bases, cdict)

In [32]:
class New(object, metaclass=MyMeta):
    pass

<class '__main__.MyMeta'>
New
(<class 'object'>,)
{'__module__': '__main__', '__qualname__': 'New'}


In [29]:
class MyMeta(type):
    def __init__(cls, name, bases, cdict):
        print(cls)
        print(name)
        print(bases)
        print(cdict)
        super().__init__(name, bases, cdict)

In [33]:
class New(object, metaclass=MyMeta):
    pass

<class '__main__.MyMeta'>
New
(<class 'object'>,)
{'__module__': '__main__', '__qualname__': 'New'}


In [43]:
class B(object):
    def __new__(cls, value):
        return super().__new__(cls)
    def __init__(self, value):
        self.value = value

In [44]:
b = B(10)

In [46]:
def decorator(cls):
    cls.attr = 10
    return cls
@decorator
class A(object):
    pass
A.__dict__['attr']

10

In [50]:
class S(object):
    __slots__ = ['a', 'b']
s = S()
s.a = 10
# s.c = 10

In [52]:
# Does not work
# class MyClass(object):
#     __slots__ = ['a', 'b']
#     a = 10
#     b = 20
class MyClass(object):
    __slots__ = ['__a', '__b']
    a = 10
    b = 10

In [70]:
def with_metaclass(meta, *bases):
    class metaclass(meta):
        __call__ = type.__call__
        __init__ = type.__init__
        def __new__(cls, name, this_bases, d):
            if this_bases is None:
                return type.__new__(cls, name, (), d)
            return meta(name, this_bases, d)
    return metaclass('temp_class', None, {})

class TypeDescriptor(object):
    def __init__(self, data_type, default_value=None):
        self.name = None
        self.data_type = data_type
        if default_value:
            self.default_value = default_value
        else:
            self.default_value = data_type
    def __get__(self, instance, cls):
        return getattr(instance, self.name)
    def __set__(self, instance, value):
        if not isinstance(value, self.data_type):
            raise TypeError
        setattr(instance, self.name, value)
        
class TypeProtected(type):
    def __new__(meta, name, bases, cdict):
        slots = []
        for key, value in cdict.items():
            if isinstance(value, TypeDescriptor):
                value.name = '__' + key
                slots.append(value.name)
        cdict['__slots__'] = slots
        return super().__new__(meta, name, bases, cdict)

In [74]:
# class Typed(with_metaclass(TypeProtected)):
#     pass

# class MyClass(Typed):
#     attr1 = TypeDescriptor(int, 1)
#     attr2 = TypeDescriptor(float, 0.2)

In [85]:
def wrapper(func):
    print(locals())
    def inner(*args, **kwargs):
        print(locals())
        return func()
    return inner

@wrapper
def func(*args, **kwargs):
    print('hello')

func()

{'func': <function func at 0x7fda04157488>}
{'kwargs': {}, 'args': (), 'func': <function func at 0x7fda04157488>}
hello
